Урок 4. Тематическое моделирование. EM-алгоритм
Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

рабочие ссылки с твитами
https://disk.yandex.ru/i/v5HM-ENiGXZVpQ
https://disk.yandex.ru/i/koR5eMCToCZS2Q

как альтернатива можно скачать данные из Роспотребнадзора
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук parse_rospotrebnadzor.ipynb
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики

In [1]:
!pip install pymorphy2
!pip install pyLDAvis
!pip install pymystem3

     ---------------------------------------- 1.7/1.7 MB 3.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136882 sha256=3dc43da07da5ea2aaf78125948f5917f673f08d3d3264081f912d40e3090b434
  Stored in directory: c:\users\uliana\appdata\local\pip\cache\wheels\90\61\ec\9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=11505d4

In [28]:
import pandas as pd
import re
import pymorphy2
from nltk.corpus import stopwords
from gensim.models import *
from gensim import corpora
import numpy as np
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [4]:
positive = pd.read_csv('positive.csv', sep = ';', usecols = [3], names = ['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('negative.csv', sep = ';', usecols = [3], names = ['text'])
negative['label'] = ['negative'] * len(negative)

In [5]:
# Объкдиняем выборки
df = pd.concat([positive, negative], ignore_index = True)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [13]:
words_regex = re.compile('\w+')

# ФОрмируем токены длиной более 3 букв
def find_words(text, regex = words_regex):
    tokens = regex.findall(text) 
    return [w for w in tokens if w.isalpha() and len(w)>= 4] 

morph = pymorphy2.MorphAnalyzer() 
stopwords_list = stopwords.words('russian') 

# Удаляем стоп слова и другие символы, не являющимися буквами
def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words] 
    return [w for w in lemmas if not w in stopwords and w.isalpha()] 

#Чистим от нецензурных слов и слов, не несущих особой смысловой нагрузки
def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    string_tokens = re.sub(r'[A-za-z]', ' ',  string_tokens) 
    string_tokens = re.sub(r'\w+ще\b', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bбл\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'значит', ' ',  string_tokens) 
    string_tokens = re.sub(r'почему', ' ',  string_tokens) 
    string_tokens = re.sub(r'свой', ' ',  string_tokens) 
    string_tokens = re.sub(r'капец', ' ',  string_tokens) 
    string_tokens = re.sub(r'который', ' ',  string_tokens)
    string_tokens = re.sub(r'это', ' ',  string_tokens)
    string_tokens = re.sub(r'весь', ' ',  string_tokens)
    string_tokens = re.sub(r'\bидио\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bпид\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхуй\w+', ' ', string_tokens)
    string_tokens = re.sub(r'\w+xу.\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+xу\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+но\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпоп\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпиз\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bгов\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bфиг\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'сука', ' ',  string_tokens)
    string_tokens = re.sub(r'очень', ' ',  string_tokens)
    string_tokens = re.sub(r'лох', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпи.де\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'нету', ' ',  string_tokens)
    string_tokens = re.sub(r'ой', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхот\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпрост\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bника\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+ть\b', ' ',  string_tokens)
    string_tokens = re.sub(r'мочь', ' ',  string_tokens)
    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

In [14]:
# Препроессинг
df['text'] = df['text'].apply(preprocess)
df.head()

,text,label
0,"[школотый, поверь, самый, общество, предмет, тип]",positive
1,"[таки, немного, похожий, мальчик, хороший]",positive
2,[испугаться],positive
3,"[угол, голод, порция]",positive
4,"[страшилка, создаться, ощущение, автор]",positive


In [17]:
# Формируем словарь уникальных токенов

dictionary = corpora.Dictionary(df['text'])

In [18]:
# Убираем редкие слова
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None)

dictionary.save('tweet.dict')

len(dictionary.keys())

8172

In [19]:
# Векторизация твитов

corpus = [dictionary.doc2bow(text) for text in df['text']]
corpora.MmCorpus.serialize('tweet.model', corpus)

In [20]:
# Итерация по словарю

mm = corpora.MmCorpus('tweet.model')

print(df['text'].values[0])
print(mm[0])

['школотый', 'поверь', 'самый', 'общество', 'предмет', 'тип']
[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]


In [21]:
# Обучение модели

lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3)

In [22]:
# Определяем тематики

lda.show_topics(num_topics=3, num_words=20, formatted=True)

[(0,
  '0.038*"сегодня" + 0.021*"школа" + 0.015*"хороший" + 0.013*"никто" + 0.013*"день" + 0.013*"жаль" + 0.010*"урок" + 0.009*"первый" + 0.008*"любимый" + 0.008*"мама" + 0.008*"каникулы" + 0.007*"жалко" + 0.007*"бояться" + 0.007*"дом" + 0.007*"вроде" + 0.006*"рука" + 0.006*"маленький" + 0.006*"фотка" + 0.006*"втор" + 0.006*"ранний"'),
 (1,
  '0.023*"человек" + 0.021*"больш" + 0.019*"время" + 0.013*"друг" + 0.010*"дом" + 0.009*"найти" + 0.008*"жизнь" + 0.008*"пока" + 0.007*"твиттер" + 0.007*"спасибо" + 0.007*"нужный" + 0.007*"вечер" + 0.006*"деньга" + 0.006*"город" + 0.006*"новый" + 0.006*"столько" + 0.005*"нога" + 0.005*"ужас" + 0.005*"получиться" + 0.005*"погода"'),
 (2,
  '0.029*"завтра" + 0.022*"день" + 0.012*"утро" + 0.011*"час" + 0.010*"год" + 0.010*"работа" + 0.010*"новый" + 0.009*"ночь" + 0.009*"голова" + 0.009*"неделя" + 0.008*"каждый" + 0.008*"скоро" + 0.008*"остаться" + 0.007*"снова" + 0.007*"дом" + 0.007*"телефон" + 0.007*"улица" + 0.007*"настроение" + 0.007*"казаться" + 0.

In [23]:
# Оценка качества модели

print(lda.log_perplexity(corpus))

-8.013653792247394


In [26]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  0.0003309134186718147


In [29]:
# Визуализация
v_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(v_data)

C:\Users\Uliana\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Результаты:

У нас получились три частые темы:
1) Школьная жизнь
2) Обсуждение повседневной жизни
3) Как будто объединяет школьную тему и повседневность. С третьей темой у меня трудности, не моугу ее классифицировать